https://www.kaggle.com/poonaml/deep-neural-network-keras-way

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train = pd.read_csv("../input/train.csv")
test= pd.read_csv("../input/test.csv")
print(train.shape)
print(test.shape)


In [ ]:
X_train = train.iloc[:,1:].values.astype('float32')
y_train = train.iloc[:,0].values.astype('int32')
X_test = test.values.astype('float32')

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28,28)
X_train.shape
for i in range(6,9):
    plt.subplot(330+(i+1))
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
X_train =  X_train.reshape(X_train.shape[0],28,28,1)
X_train.shape

In [ ]:
X_test = X_test.reshape(X_test.shape[0],28,28,1)
X_test.shape

In [ ]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [ ]:
def standardize(x):
    return (x-mean_px)/std_px

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes


In [ ]:
seed = 43
np.random.seed(seed)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Lambda, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Lambda(standardize, input_shape = (28,28,1)))
model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))
print("input shape ", model.input_shape)
print("output shape ", model.output_shape)

In [ ]:
from keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr=0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.preprocessing import image
gen = image.ImageDataGenerator()

In [ ]:
from sklearn.model_selection import train_test_split
X = X_train
y = y_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 10, random_state=42)
batches = gen.flow(X_train, y_train, batch_size = 64) 
val_batches = gen.flow(X_val, y_val, batch_size=64)

In [ ]:
history = model.fit_generator(generator = batches, steps_per_epoch = batches.n, epochs=3, validation_data=val_batches, validation_steps=val_batches.n)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1,len(loss_values)+1)
plt.plot(epochs, loss_values, 'bo')
plt.plot(epochs, val_loss_values, 'b+')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
def get_fc_model():
    model = Sequential([Lambda(standardize, input_shape=(28,28,1)),
                       Flatten(),
                       Dense(512, activation='relu'),
                       Dense(10, activation='softmax')])
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
fc = get_fc_model()
fc.optimizer.lr = 0.01

In [ ]:
np.set_printoptions(precision=6)

In [ ]:
history=fc.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                    validation_data=val_batches, validation_steps=val_batches.n)

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D
def get_cnn_model():
    model = Sequential([
        Lambda(standardize,input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,(3,3), activation='relu'),
        Convolution2D(64,(3,3), activation = 'relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(),loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    

In [ ]:
model = get_cnn_model()
model.optimizer.lr = 0.01

In [ ]:
history = model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, validation_data=val_batches, validation_steps=val_batches.n)

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)
submissions = pd.DataFrame({"ImageId":list(range(1,len(predictions)+1)),"Label":predictions})
submissions.to_csv("DR.csv",index=False, header=True)